<a href="https://colab.research.google.com/github/smannan/LIFXBulbAnalysis/blob/main/LSTMEnergyModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
import numpy as np
import pandas as pd

In [31]:
n_features = 24

In [32]:
filename = '/content/drive/Shared drives/CMPE295B/JoinedPGEBulbData/2021-08-26_13_2021-09-26_22_hourly.csv'
bulb_data = pd.read_csv(filename, infer_datetime_format = True).drop(['Unnamed: 0'], axis=1, inplace=False)
bulb_data['timestamp'] = pd.to_datetime(bulb_data['timestamp'])

In [33]:
bulb_data.head()

,index,timestamp,brightness,kelvin,kWH,cost,usage,percent_usage_by_bulb,cost_per_bulb
0,0,2021-08-26 13:00:00,1.0,2700.0,0.0085,0.09,0.35,2.428571,0.002186
1,1,2021-08-26 14:00:00,1.0,2700.0,0.0085,0.11,0.38,2.236842,0.002461
2,2,2021-08-26 15:00:00,1.0,2700.0,0.0085,0.09,0.34,2.500000,0.002250
3,3,2021-08-26 16:00:00,1.0,2700.0,0.0085,0.08,0.29,2.931034,0.002345
4,4,2021-08-26 17:00:00,1.0,2700.0,0.0085,0.11,0.40,2.125000,0.002338


In [34]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)

	return np.array(X), np.array(y)

In [38]:
X, y = split_sequence(bulb_data['cost_per_bulb'], n_features)

In [39]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
X_reshaped = X.reshape((X.shape[0], X.shape[1], n_features))

ValueError: ignored

In [42]:
print (X)

[[0.00218571 0.00246053 0.00225    ... 0.         0.         0.        ]
 [0.00246053 0.00225    0.00234483 ... 0.         0.         0.        ]
 [0.00225    0.00234483 0.0023375  ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
